In [1]:
import torch
from peft import (
    PeftConfig,
    PeftModel
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

In [2]:
PEFT_MODEL = "./models/llama-2-7b-chat-hf-midjourney-finetuned-peft/"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

# %%
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

# %%
device = "cuda:0"

prompt = """
<human>: midjourney prompt for a boy running in the snow
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

<human>: midjourney prompt for a boy running in the snow
<assistant>: Boy running in the snow, snow-covered field, matte painting, Cinematic, 8k, hyper-realistic, photorealistic --ar 16:9 --uplight --no dof --octane render --bitmooth --statue of liberty in foreground --trending on Artstation --shapemating --ar 16:9 --cubism --geometric --muted colors --ar 16:9 --photorealistic --atmosphere --matthew wilder --m Machine learning --sharpen --res 8k --no volumetric lighting --octane -- 8k --houdini --ar 16:9 --high detailed --realistic --atmosphere --lighting --mood --night light --warm light --artistic --octane render --ar 16:9 --no fog --ar 16:
